<left> <img src="https://radicalbit.ai/wp-content/uploads/2024/02/radicalbit-logo-bk.png" width="400" /> </left>


## Radicalbit Quickstart: Monitor a Regression Model

### Introduction
This guide provides instructions on how to monitor a ML solution with the Radicalbit OS Platform, through the Python SDK (https://pypi.org/project/radicalbit-platform-sdk/).



In [2]:
from radicalbit_platform_sdk.client import Client
from radicalbit_platform_sdk.models import (
    AwsCredentials,
    CreateModel,
    DataType,
    ModelType,
    ColumnDefinition,
    OutputType,
    Granularity,
)

from datetime import datetime 
import pandas as pd


### Create the Client
In order to communicate with the platform, you need to create the client and indicate the URL where the UI will be available.
Remember that before you need to launch the platform following the instructions in the README.md (https://github.com/radicalbit/radicalbit-ai-monitoring/blob/main/README.md).

In [4]:
# Create the Client
base_url = "http://localhost:9000"
client = Client(base_url)


### The reference dataset
The reference dataset is the name we use to indicate the batch that contains the information we desire to have constantly (or we expect to have) over time. It could be the training set or a chunck of production data where the model has had good performances.

To use the radicalbit-ai-monitoring platform, you need first to prepare your reference data, which should include the following information:

- **Variables**: The list of features used by the model as well as other information like metadata produced by the system
- **Outputs**: The fields returned by the model after the inference. Usually, they are probabilities, a predicted class or numbers.
- **Target**: the ground truth used to validate predictions and evaluate the model quality
- **Timestamp**: The timestamp field used to aggregate data over selected windows.

In this example we will use a dataset built to compute the number of rings, and therefore the age, of an abalone.



> **_Dataset license:_**  Nash,Warwick, Sellers,Tracy, Talbot,Simon, Cawthorn,Andrew, and Ford,Wes. (1995). Abalone. UCI Machine Learning Repository. https://doi.org/10.24432/C55C7W. Adapted by Radicalbit.


In [8]:
reference_path = "../data/regression/regression_abalone_reference.csv"
reference = pd.read_csv(reference_path)
reference.head(3)


,Sex,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight,ground_truth,prediction,pred_id,timestamp
0,M,0.66,0.525,0.200,1.4890,0.6065,0.3795,0.4210,11,11,e73e8874-9f00-4d78-ab43-9b6840172e38,2024-01-10 08:00:00
1,F,0.64,0.505,0.165,1.2235,0.5215,0.2695,0.3600,2,2,721e6420-6086-4444-b1be-222404bd1194,2024-01-10 08:00:02
2,M,0.65,0.515,0.180,1.4630,0.6580,0.3135,0.4115,2,2,4b6a472e-a124-40fe-832d-71422eb47671,2024-01-10 08:00:04


### Create the Model
The next step requires the Model creation.
Here, you have to specify the following information:

- **name:** The name of the model
- **model_type:** The type of the model
- **data_type:** It explains the data type used by the model
- **granularity:** The window used to calculate aggregated metrics with the current data
- **features:** A list column representing the features set
- **outputs:** An OutputType definition to explain the output of the model
- **target:** The column used to represent the model's target
- **timestamp:** The column used to store the when prediction was done
- **frameworks:** An optional field to describe the frameworks used by the model
- **algorithm:** An optional field to explain the algorithm used by the model

In [11]:
# Create the Model
model = CreateModel(
    name=f"Model-{str(datetime.now()).replace(' ', '-').replace(':', '-').split('.')[0]}",
    modelType=ModelType.REGRESSION,
    dataType=DataType.TABULAR,
    granularity=Granularity.DAY,
    description="This is a model to compute the number of rings, and therefore the age, of an abalone.",
    features=[
        ColumnDefinition(name="Sex", type="string"),
        ColumnDefinition(name="Length", type="float"),
        ColumnDefinition(name="Diameter", type="float"),
        ColumnDefinition(name="Height", type="float"),
        ColumnDefinition(name="Whole_weight", type="float"),
        ColumnDefinition(name="Shucked_weight", type="float"),
        ColumnDefinition(name="Viscera_weight", type="float"),
        ColumnDefinition(name="Shell_weight", type="float")        
    ],
    outputs=OutputType(
        prediction=ColumnDefinition(name="prediction", type="int"),
        output=[
            ColumnDefinition(name="prediction", type="int"),
            ColumnDefinition(name="pred_id", type="string")
        ],
    ),
    target=ColumnDefinition(name="ground_truth", type="int"),
    timestamp=ColumnDefinition(name="timestamp", type="datetime"),
)

model = client.create_model(model)


In [13]:
print(model.name())
print(model.uuid())
print(model.data_type())
print(model.description())


Model-2024-07-09-15-16-07
13026e41-b28b-466c-9983-78004cd0d2b6
DataType.TABULAR
This is a model to compute the number of rings, and therefore the age, of an abalone.


After this action, go to the platform to see:

 - In the **Overview** section, you will see the generated schema of Variables and Outputs

### Load the reference dataset
Once the model has been created, you are ready to upload your reference dataset into the platform. All you need is to run the following code, in which you have to specify the path of your file and set up your AWS credentials as indicated here. 
In this case, we use Minio as a substitute for a real AWS. 



In [17]:
# load the reference dataset
ref = model.load_reference_dataset(
    file_name=reference_path,
    bucket="test-bucket",
    aws_credentials=AwsCredentials(
        access_key_id="minio",
       secret_access_key="minio123",
       default_region="us-east-1",
        endpoint_url="http://localhost:9090"
    )
)


After this action, go to the platform to see:

 - In the **Overview/Summary** section, you will see a summary of your data (missing values, number of rows or columuns and other)y
- in the **Reference** section you will see information about Data Quality and Model Quality

### Load the current dataset
The last step regards the current data uploading. The current dataset is the name we use to indicate the batch that contains fresh information, for example, the most recent production data, predictions or ground truths. We expect that it has the same characteristics (statistical properties) as the reference, which indicates that the model has the performance we expect and there is no drift in the data.
As you can see, the code is pretty similar to the reference one. 

In [21]:
current1_path = "../data/regression/regression_abalone_current1.csv"

# load the current dataset
cur1 = model.load_current_dataset(
    file_name=current1_path,
    correlation_id_column="pred_id",
    bucket="test-bucket",
    aws_credentials=AwsCredentials(
        access_key_id="minio",
       secret_access_key="minio123",
       default_region="us-east-1",
        endpoint_url="http://localhost:9090"
    )
)


After this action, go to the platform to see:

 - in the **Current** section you will see information about Data Quality and Model Quality compared to the Reference information
 - in the **Current/Import** section you will see and browse your uploaded current data